In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
from scipy.stats import mode

temperature = pd.read_csv("기온.csv", encoding="cp949")
rainfall = pd.read_csv("강수량.csv", encoding="cp949")
wind = pd.read_csv("바람.csv", encoding="cp949")
humidity = pd.read_csv("습도.csv", encoding="cp949")
sunshine = pd.read_csv("일조일사.csv", encoding="cp949")

temperature = temperature.drop([temperature.columns[0], temperature.columns[1]], axis=1)
rainfall = rainfall.drop([rainfall.columns[0], rainfall.columns[1]], axis=1)
wind = wind.drop([wind.columns[0], wind.columns[1]], axis=1)
humidity = humidity.drop([humidity.columns[0], humidity.columns[1]], axis=1)
sunshine = sunshine.drop([sunshine.columns[0], sunshine.columns[1]], axis=1)

temperature.rename(columns = {"\t최고기온시각": "최고기온시각"}, inplace=True)
wind.rename(columns = {"\t최대풍속시각":"최대풍속시각", "\t최대순간풍속시각": "최대순간풍속시각"}, inplace=True)

merge_data = pd.merge(temperature, rainfall, on='일시')
merge_data = pd.merge(merge_data, wind, on='일시')
merge_data = pd.merge(merge_data, humidity, on='일시')
merge_data = pd.merge(merge_data, sunshine, on='일시')
merge_data = merge_data[:2158]

merge_data.to_csv("observed.csv", mode='w', encoding='cp949')
observed = pd.read_csv("observed.csv", encoding='cp949')
observed = observed.drop([observed.columns[0]], axis=1) # 저장하면서 index열이 생김. 이를 지우기 위함.

weather = pd.read_csv("기상(타겟).csv", encoding="cp949", low_memory=False) # low_memory = 경고 무시용

weather = weather[weather['지역']=='전라남도']
weather = weather.drop(weather.columns[0], axis=1)

weather = weather.sort_values(by=['예보시각'])
weather = weather.reset_index(drop=True)

forecast_score = {'맑음':0,
                  '구름조금':1, '구름많음':1.1,
                  '흐림':2,
                  '구름많고 소나기':3, '구름많고 비':3.1, '구름많고 비/눈':3.2, '구름많고 눈/비':3.2, '구름많고 눈':3.3,
                  '흐리고 비':4, '흐리고 눈/비':4.1, '흐리고 비/눈':4.1, '흐리고 눈':4.2}
re_score = {v:k for k,v in forecast_score.items()}
from scipy.stats import mode

def select_forecast(date):
    temp = 0
    for forecast in date:
        if forecast_score[forecast] > temp:
            temp = forecast_score[forecast]
    return re_score[temp]
weather_copy = weather.copy()
for i, w in enumerate(weather_copy['예보시각']):
    weather_copy['예보시각'][i] = w[:10]
    
weather_copy = weather_copy.drop_duplicates(['예보시각'])
weather_copy = weather_copy.reset_index(drop=True)
weather_copy['예보'] = ''

w=0
start = 0
for i in range(len(weather)):
    if weather_copy["예보시각"][w] not in weather["예보시각"][i]:
        weather_copy["예보"][w] = select_forecast(weather["예보"][start:i])
        w += 1
        start = i
        
weather = weather_copy

weather = weather.drop([weather.columns[0]], axis=1)

weather.to_csv("forecast.csv", mode='w', encoding='cp949')
forecast = pd.read_csv("forecast.csv", encoding='cp949')
forecast = forecast.drop([forecast.columns[0]], axis=1) # 저장하면서 index열이 생김. 이를 지우기 위함.

forecast.rename(columns = {"예보시각": "일시"}, inplace=True)
df = pd.merge(observed, forecast, on='일시')
df.to_csv("df.csv", encoding='cp949')
df = pd.read_csv("df.csv", encoding='cp949')
df = df.drop([df.columns[0]], axis=1) # 저장하면서 index열이 생김. 이를 지우기 위함.

df


,일시,평균기온(℃),최고기온(℃),최고기온시각,최저기온(℃),최저기온시각,일교차,강수량(mm),1시간최다강수량(mm),1시간최다강수량시각,...,최대풍속시각,최대순간풍속(m/s),최대순간풍속풍향(deg),최대순간풍속시각,평균습도(%rh),최저습도(%rh),일조합,일조율,일사합,예보
0,2016-06-01,22.9,28.4,15:10,18.5,5:31,9.9,NaN,NaN,NaN,...,9:14,9.1,110.0,9:08,59.1,40.0,12.1,83.4,30.03,구름많음
1,2016-06-02,22.5,29.3,15:02,15.1,5:42,14.2,NaN,NaN,NaN,...,17:08,6.4,290.0,17:21,51.1,24.0,10.7,73.8,28.52,구름조금
2,2016-06-03,22.7,29.0,15:37,17.1,5:46,11.9,NaN,NaN,NaN,...,17:28,5.7,160.0,0:12,60.1,29.0,10.0,69.0,27.49,구름많음
3,2016-06-04,19.9,22.0,14:14,17.1,23:55,4.9,0.4,0.2,NaN,...,14:26,4.4,290.0,12:54,77.0,60.0,0.0,0.0,6.31,흐리고 비
4,2016-06-05,22.9,30.3,15:58,14.6,5:14,15.7,NaN,NaN,NaN,...,13:42,10.4,110.0,14:07,59.3,33.0,12.5,86.2,30.93,구름많음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,2021-11-23,1.8,3.8,14:33,-1.5,7:30,5.3,0.2,NaN,NaN,...,5:25,6.3,70.0,5:29,76.6,54.0,3.7,37.0,5.68,흐리고 비/눈
1246,2021-11-24,6.1,12.4,13:56,-0.1,7:22,12.5,0.0,NaN,NaN,...,11:26,7.4,250.0,11:20,66.0,39.0,8.6,86.0,10.72,흐리고 비/눈
1247,2021-11-25,8.7,15.8,13:45,3.9,23:53,11.9,NaN,NaN,NaN,...,14:47,9.0,360.0,14:43,66.5,34.0,8.3,83.0,10.66,구름많음
1248,2021-11-26,5.4,11.7,14:41,-0.3,7:44,12.0,NaN,NaN,NaN,...,15:55,5.9,50.0,14:18,62.5,31.0,8.7,87.0,11.44,구름많음
